In [1]:
# !pip install simpy

In [15]:
"""
Simple simulation of several bike rental stations

Stations are modeled with containers so bikes can be returned
to a station different from where it was rented from

programer:  Michael R. Gibbs
"""

import simpy
import random
import csv
import os, sys

# scenario attributes
station_names = ['A','B']
rent_probs = [.9,.1]
return_probs = [.5,.5]
bikes_per_station = 5


# def check_header(csvfile):
#   """Checks if a header already exists in a CSV file.

#   Args:
#     csv_file: The path to the CSV file.

#   Returns:
#     True if the header exists, False otherwise.
#   """

#   with open(csvfile, 'r') as f:
#     reader = csv.reader(f)
#     header = next(reader)

#   return header is not None

def rent_proc(env, id, station_names, rent_probs, return_probs, station_map):
    """
    Models the process of:
    selecting a station
    renting a bike
    using a bike
    returning a bike (can be different station)
    """

    #select a station
    csv_file = 'event_generator.csv'

    file_exists = os.path.isfile(csv_file)

    with open('event_generator.csv', 'a', newline='') as csvfile:
        
        name = random.choices(station_names,weights=rent_probs)
        name = name[0]
        station = station_map[name]

        print(f'{env.now}: id:{id} has arrived at station {name} q-len:{len(station.get_queue)} and {station.level} bikes')
        # row_data = [env.now, id, 'has arrived at station', name, len(station.get_queue), str(station.level)+' bikes']
        arrived_event = {'env': env.now, 'id' : id, 'event' : 'has arrived at station ', 'station' : name, 'q_len' : len(station.get_queue),
                   'bike_count' : str(station.level)+' bikes'}

        # Create a CSV writer object
        # writer = csv.writer(csvfile)
        writer = csv.DictWriter(csvfile, fieldnames=['env', 'id', 'event', 'station', 'q_len', 'bike_count'])

        # Write the header row
        # if not check_header(csvfile):
        # writer.writerow(header_data)
        if not file_exists:
            writer.writeheader()

        # Write the rows of data
        writer.writerow(arrived_event)

        # get a bike
        yield station.get(1)

        print(f'{env.now}: id:{id} has rented bike at station {name} q-len:{len(station.get_queue)} and {station.level} bikes')
        rented_event = {'env': env.now, 'id' : id, 'event' : 'has rented bike at station ', 'station' : name, 'q_len' : len(station.get_queue),
                   'bike_count' : str(station.level)+' bikes'}

        writer.writerow(rented_event)
        
        

        # use bike
        yield env.timeout(random.triangular(1,5,3))

        # return bike
        name = random.choices(station_names,weights=return_probs)
        name = name[0]
        station = station_map[name]

        yield station.put(1)

        print(f'{env.now}: id:{id} has returned bike at station {name} q-len:{len(station.get_queue)} and {station.level} bikes')

        returned_event = {'env': env.now, 'id' : id, 'event' : 'has returned bike at station ', 'station' : name, 'q_len' : len(station.get_queue),
                   'bike_count' : str(station.level)+' bikes'}

        writer.writerow(returned_event)

    csvfile.close()

def gen_arrivals(env, station_names, rent_probs, return_probs, station_map):
    """
    Generates arrivales to the rental stations
    """

    cnt = 0

    while True:
        yield env.timeout(random.expovariate(2.5))
        cnt += 1
        env.process(rent_proc(env,cnt,station_names,rent_probs,return_probs, station_map))

# set up
env = simpy.Environment()

# create station based on name list
cap = len(station_names) * bikes_per_station
station_map = {
    name: simpy.Container(env, init=bikes_per_station, capacity=cap)
    for name in station_names
}

# start generation arrivals
env.process(gen_arrivals(env, station_names, rent_probs, return_probs, station_map))

# start sim
env.run(100)



0.6916729882282272: id:1 has arrived at station A q-len:0 and 5 bikes
0.6916729882282272: id:1 has rented bike at station A q-len:0 and 4 bikes
0.8868081266711825: id:2 has arrived at station A q-len:0 and 4 bikes
0.8868081266711825: id:2 has rented bike at station A q-len:0 and 3 bikes
1.2587592806860033: id:3 has arrived at station A q-len:0 and 3 bikes
1.2587592806860033: id:3 has rented bike at station A q-len:0 and 2 bikes
1.4880963002078595: id:4 has arrived at station A q-len:0 and 2 bikes
1.4880963002078595: id:4 has rented bike at station A q-len:0 and 1 bikes
2.1696672870741116: id:2 has returned bike at station B q-len:0 and 6 bikes
2.208980371369358: id:5 has arrived at station A q-len:0 and 1 bikes
2.208980371369358: id:5 has rented bike at station A q-len:0 and 0 bikes
2.2097248136478833: id:6 has arrived at station A q-len:0 and 0 bikes
2.24690293698876: id:7 has arrived at station A q-len:1 and 0 bikes
2.3666996624387426: id:8 has arrived at station B q-len:0 and 6 bike